## Imports

In [49]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import  mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

df_raw = pd.read_csv('../data/treino.csv')
df_test = pd.read_csv('../data/teste.csv')

In [50]:
def data_transformation(df):
    df = df_test.copy()
    df.columns = ['id', 'num_fotos', 'marca', 'modelo', 'versao', 'ano_de_fabricacao',
           'ano_modelo', 'odometro', 'cambio', 'num_portas', 'tipo', 'blindado',
           'cor', 'tipo_vendedor', 'cidade_vendedor', 'estado_vendedor',
           'tipo_anuncio', 'entrega_delivery', 'troca', 'elegivel_revisao',
           'aceita_troca', 'dono_unico',
           'todas_revisoes_concessionaria',
           'ipva_pago', 'licenciado',
           'garantia_de_fabrica',
           'todas_revisoes_agenda',
           'alienado']

    df['num_fotos'] = df['num_fotos'].fillna(0)

    df = df.drop('alienado', axis=1)

    na_cols = df.columns[-8:]

    for col in na_cols:
        df[col] = np.where(df[col].isna(), 0, 1)

    df[df.T.tail(8).index] = df[df.T.tail(8).index].astype('int64')

    df['ano_modelo'] = df['ano_modelo'].astype('int64')
    df['num_fotos'] = df['num_fotos'].astype('int64')

    X = df.copy().drop('id', axis=1)

    #drop cidades
    X = X.drop('cidade_vendedor', axis=1)

    #get uf
    X['estado_vendedor'] = X['estado_vendedor'].apply(lambda x: x[-3:-1])

    #numeric transformation
    num_columns = X.select_dtypes(exclude = 'object').columns
    cat_columns = X.select_dtypes('object').columns
    mms = pp.MinMaxScaler()

    for col in num_columns:
        X[col] = mms.fit_transform(X[[col]] )


    target_marca = pd.read_pickle('../parameters/marca_encode.pkl')
    target_modelo = pd.read_pickle('../parameters/modelo_encode.pkl')
    target_versao = pd.read_pickle('../parameters/versao_encode.pkl')
    target_tipo =  pd.read_pickle('../parameters/tipo_encode.pkl')
    frequency_cor = pd.read_pickle('../parameters/cor_encode.pkl')
    frequency_tipo_anuncio = pd.read_pickle('../parameters/tipo_anuncio_encode.pkl')
    map_cor = {'Preto':'preto', 'Branco':'branco', 'Prata':'prata', 'Cinza':'cinza', 'Dourado':'outros', 'Vermelho':'outros', 'Azul':'outros',
           'Verde':'outros'}

    map_regiao={'SP':'sudeste','RS':'sul','MG':'sudeste','PR':'sul','RJ':'sudeste','MA':'nordeste','SC':'sul','AL':'nordeste','BA':'nordeste','GO':'centro_oeste','RN':'nordeste','PE':'nordeste','MT':'centro_oeste','PA':'norte','CE':'nordeste','AM':'nordeste','ES':'sudeste','RO':'norte','PB':'nordeste','TO':'norte','AC':'norte','SE':'nordeste','MS':'centro_oeste','RR':'norte','PI':'nordeste'}


    map_cambio = {'Automática': 'auto', 'Manual' :'manual', 'CVT' :'auto', 'Automatizada': 'auto', 'Semi-automática': 'semi',
           'Automatizada DCT' : 'auto', 'Automática Sequencial' : 'auto'}


    #map categorical features
    X['cor'] = X['cor'].map(map_cor)
    X['cor'] = X['cor'].map(frequency_cor)
    X['tipo_anuncio'] = X['tipo_anuncio'].map(frequency_tipo_anuncio)
    X['estado_vendedor'] = X['estado_vendedor'].map(map_regiao)
    X['marca'] = X['marca'].map(target_marca)
    X['modelo'] = X['modelo'].map(target_modelo)
    X['versao'] = X['versao'].map(target_versao)
    X['tipo'] = X['tipo'].map(target_tipo)
    X['cambio'] = X['cambio'].map(map_cambio)


    X = pd.get_dummies(X, columns=['cor', 'blindado', 'estado_vendedor', 'cambio', 'tipo_vendedor'])

    return X
    
    

## Data Descriptions

In [51]:
df1 = df_raw.copy()

In [52]:
df_raw.columns

Index(['ID', 'num_fotos', 'marca', 'modelo', 'versao', 'ano_de_fabricacao',
       'ano_modelo', 'odometro', 'cambio', 'num_portas', 'tipo', 'blindado',
       'cor', 'tipo_vendedor', 'cidade_vendedor', 'estado_vendedor',
       'tipo_anuncio', 'entrega_delivery', 'troca', 'elegivel_revisao',
       'attr_veiculo_aceita_troca', 'attr_veiculo_único_dono',
       'attr_veiculo_todas_as_revisões_feitas_pela_concessionária',
       'attr_veiculo_ipva_pago', 'attr_veiculo_licenciado',
       'attr_veiculo_garantia_de_fábrica',
       'attr_veiculo_todas_as_revisões_feitas_pela_agenda_do_carro',
       'attr_veiculo_alienado', 'preco'],
      dtype='object')

In [53]:
# rename columns
df1.columns = ['id', 'num_fotos', 'marca', 'modelo', 'versao', 'ano_de_fabricacao',
       'ano_modelo', 'odometro', 'cambio', 'num_portas', 'tipo', 'blindado',
       'cor', 'tipo_vendedor', 'cidade_vendedor', 'estado_vendedor',
       'tipo_anuncio', 'entrega_delivery', 'troca', 'elegivel_revisao',
       'aceita_troca', 'dono_unico',
       'todas_revisoes_concessionaria',
       'ipva_pago', 'licenciado',
       'garantia_de_fabrica',
       'todas_revisoes_agenda',
       'alienado', 'preco']

### Data dimension / types / nas

In [54]:
df_raw.shape

(39446, 29)

In [55]:
df_raw.dtypes

ID                                                             object
num_fotos                                                     float64
marca                                                          object
modelo                                                         object
versao                                                         object
ano_de_fabricacao                                               int64
ano_modelo                                                    float64
odometro                                                      float64
cambio                                                         object
num_portas                                                      int64
tipo                                                           object
blindado                                                       object
cor                                                            object
tipo_vendedor                                                  object
cidade_vendedor     

In [56]:
df1.isna().sum() / len(df1)

id                               0.000000
num_fotos                        0.006008
marca                            0.000000
modelo                           0.000000
versao                           0.000000
ano_de_fabricacao                0.000000
ano_modelo                       0.000000
odometro                         0.000000
cambio                           0.000000
num_portas                       0.000000
tipo                             0.000000
blindado                         0.000000
cor                              0.000000
tipo_vendedor                    0.000000
cidade_vendedor                  0.000000
estado_vendedor                  0.000000
tipo_anuncio                     0.000000
entrega_delivery                 0.000000
troca                            0.000000
elegivel_revisao                 0.000000
aceita_troca                     0.258987
dono_unico                       0.647670
todas_revisoes_concessionaria    0.689728
ipva_pago                        0

### Stats

In [57]:
df_raw.select_dtypes('object').describe().T

,count,unique,top,freq
ID,39446,39446,105869496634249611881157692933406072990,1
marca,39446,41,VOLKSWAGEN,6140
modelo,39446,473,COMPASS,1967
versao,39446,2058,1.6 16V FLEX ALLURE PACK 4P AUTOMÁTICO,1825
cambio,39446,7,Automática,30073
tipo,39446,8,Sedã,21910
blindado,39446,2,N,39128
cor,39446,8,Branco,27919
tipo_vendedor,39446,2,PF,23857
cidade_vendedor,39446,648,São Paulo,8198


In [58]:
df_raw.select_dtypes(exclude='object').describe().T

,count,mean,std,min,25%,50%,75%,max
num_fotos,39209.0,10.323829,3.481065,8.000000,8.000000,8.000000,14.000000,2.100000e+01
ano_de_fabricacao,39446.0,2016.748137,4.084762,1985.000000,2015.000000,2018.000000,2019.000000,2.022000e+03
ano_modelo,39446.0,2017.807154,2.675334,1997.000000,2016.000000,2018.000000,2020.000000,2.023000e+03
odometro,39446.0,58382.243371,32542.793054,100.000000,31235.250000,57244.500000,81925.750000,3.900650e+05
num_portas,39446.0,3.941135,0.337210,2.000000,4.000000,4.000000,4.000000,4.000000e+00
attr_veiculo_alienado,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
preco,39446.0,133385.874348,82384.971901,7258.159943,76774.321715,114307.970618,163987.004263,1.359813e+06


### Change types / fillna

In [59]:
# NAS
# num_fotos                        0.006008
# aceita_troca                     0.258987
# dono_unico                       0.647670
# todas_revisoes_concessionaria    0.689728
# ipva_pago                        0.335218
# licenciado                       0.461289
# garantia_de_fabrica              0.853268
# todas_revisoes_agenda            0.799650
# alienado                         1.000000


df1['num_fotos'] = df1['num_fotos'].fillna(0)

df1 = df1.drop('alienado', axis=1)

na_cols = df1.columns[-8:-1]


for col in na_cols:
    df1[col] = np.where(df1[col].isna(), 0, 1)

df1[df1.T.tail(8).index] = df1[df1.T.tail(8).index].astype('int64')

df1['ano_modelo'] = df1['ano_modelo'].astype('int64')
df1['num_fotos'] = df1['num_fotos'].astype('int64')



## Data Filtering

In [60]:
# df1[(df1['marca']=='CHEVROLET') & (df1['modelo']=='ONIX')].sort_values('ano_modelo')


## Feature Engineering

## EDA

## Data Preparation

In [61]:
X = df1.copy().drop('id', axis=1)

#one hot
#frequency
#target
#label 




#drop cidades
X = X.drop('cidade_vendedor', axis=1)

#get uf
X['estado_vendedor'] = X['estado_vendedor'].apply(lambda x: x[-3:-1])

#numeric transformation
num_columns = X.select_dtypes(exclude = 'object').columns
cat_columns = X.select_dtypes('object').columns
mms = pp.MinMaxScaler()

for col in num_columns:
    X[col] = mms.fit_transform(X[[col]] )
    

target_marca = X.groupby('marca')['preco'].mean()
target_modelo = X.groupby('modelo')['preco'].mean()
target_versao = X.groupby('versao')['preco'].mean()
target_tipo = X.groupby('tipo')['preco'].mean()
frequency_cor = X.groupby('cor')['preco'].count() / len(X)
frequency_tipo_anuncio = X.groupby('tipo_anuncio')['preco'].count() / len(X)

map_cor = {'Preto':'preto', 'Branco':'branco', 'Prata':'prata', 'Cinza':'cinza', 'Dourado':'outros', 'Vermelho':'outros', 'Azul':'outros',
       'Verde':'outros'}

map_regiao={'SP':'sudeste','RS':'sul','MG':'sudeste','PR':'sul','RJ':'sudeste','MA':'nordeste','SC':'sul','AL':'nordeste','BA':'nordeste','GO':'centro_oeste','RN':'nordeste','PE':'nordeste','MT':'centro_oeste','PA':'norte','CE':'nordeste','AM':'nordeste','ES':'sudeste','RO':'norte','PB':'nordeste','TO':'norte','AC':'norte','SE':'nordeste','MS':'centro_oeste','RR':'norte','PI':'nordeste'}


map_cambio = {'Automática': 'auto', 'Manual' :'manual', 'CVT' :'auto', 'Automatizada': 'auto', 'Semi-automática': 'semi',
       'Automatizada DCT' : 'auto', 'Automática Sequencial' : 'auto'}


#map categorical features
X['cor'] = X['cor'].map(map_cor)
X['cor'] = X['cor'].map(frequency_cor)
X['tipo_anuncio'] = X['tipo_anuncio'].map(frequency_tipo_anuncio)
X['estado_vendedor'] = X['estado_vendedor'].map(map_regiao)
X['marca'] = X['marca'].map(target_marca)
X['modelo'] = X['modelo'].map(target_modelo)
X['versao'] = X['versao'].map(target_versao)
X['tipo'] = X['tipo'].map(target_tipo)
X['cambio'] = X['cambio'].map(map_cambio)


X = pd.get_dummies(X, columns=['cor', 'blindado', 'estado_vendedor', 'cambio', 'tipo_vendedor'])


target_marca.to_pickle('../parameters/marca_encode.pkl')
target_modelo.to_pickle('../parameters/modelo_encode.pkl')
target_versao.to_pickle('../parameters/versao_encode.pkl')
frequency_cor.to_pickle('../parameters/cor_encode.pkl')
target_tipo.to_pickle('../parameters/tipo_encode.pkl')
frequency_tipo_anuncio.to_pickle('../parameters/tipo_anuncio_encode.pkl')


frequency_tipo_anuncio = X.groupby('tipo_anuncio')['preco'].count() / len(X)

In [62]:
y = X['preco'].copy()
X = X.drop('preco', axis=1)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [64]:
# model definition
lr = LinearRegression()

# model train
lr.fit(X_train, y_train)

# predict
y_hat = lr.predict(X_test)

# performance
print(mean_absolute_error(y_test, y_hat))
print(np.sqrt(mean_squared_error(y_test, y_hat)))

0.019916349729043287
0.02930152025199432


In [35]:
comparacao = pd.DataFrame()
comparacao['real'] = y_test
comparacao['predicao'] = y_hat

In [37]:
comparacao['diff'] = comparacao['real'] -  comparacao['predicao']

In [39]:
(comparacao['diff']**2).sum()

7.2146888612085815

## ML Models

In [73]:
X_test = data_transformation(df_test)


# predict
y_hat = lr.predict(X_test.fillna(0))

teste = pd.DataFrame()
teste['ID'] = df_test['ID'] 
teste['preco'] = y_hat



In [74]:
teste

,ID,preco
0,24813264385557040124808779273028388499,0.046094
1,295636316453795508942188530111300065666,0.084627
2,101258309166227950735244624080888109884,0.063011
3,28348734455782469411126661985772047409,0.051677
4,193163160502972147671913739170248305797,0.070204
...,...,...
39441,238233399351588823822117090805568390727,0.016240
39442,64621912306231118962468441892654163025,0.110081
39443,100311033226508317456901122129284293382,0.104387
39444,217317181330151694133399005110777689124,0.249858
